In [3]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="mixtral-8x7b-32768"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong on July 20, 1969, as part of the Apollo 11 mission. Armstrong's famous words, "That's one small step for man, one giant leap for mankind," were spoken as he became the first human to set foot on the lunar surface.


In [56]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.adidas-group.com/jobs?brand=&team=Marketing%20%26%20Communications%3BDesign%3BDigital%3BProduct%20Development%3BInformation%20Technology%3BEngineering&type=&keywords=&location=%5B%5D&sort=&locale=en&offset=0&utm_source=linkedin&utm_medium=video&utm_campaign=eb")
page_data = loader.load().pop().page_content
print(page_data)






                adidas Careers – Through sport, we have the power to change lives. All Job Openings
        















































English
German




The Company


Teams




                                    Makers
                                


Design


Digital


Engineering


Marketing & Communications


Product Development


Production


Research & Development





                                    Growers
                                


Accounting & Finance


Administrative


Audit


Customer Service


Facilities & Services


Human Resources


Tech


Legal / Social Environmental Affairs


Sales


Sourcing


Strategy


Supply Chain Management





                                    Retailers
                                


Retail Back Office


Retail Store


Our Stores


Retail Career Opportunities


 





                                    Pupils
                                


Pupil Internship


Apprenticeship Program


Dual Study 

In [57]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [58]:
from langchain_core.output_parsers import JsonOutputParser


json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

if isinstance(json_res, list):
    json_dict = {i: json_res[i] for i in range(len(json_res))}  
    print(json_dict)
else:
    print("json_res is not a list.")




{0: {'role': 'Assistant Manager Buying eCom - TEMP 6 months', 'experience': 'Not specified', 'skills': ['Digital'], 'description': 'Not provided'}, 1: {'role': 'Manager Wholesale Activations EM South - Maternity Cover', 'experience': 'Not specified', 'skills': ['Marketing & Communications'], 'description': 'Not provided'}, 2: {'role': 'Lifestyle Marketing Specialist', 'experience': 'Not specified', 'skills': ['Marketing & Communications'], 'description': 'Not provided'}, 3: {'role': 'Sr Specialist Marketing Operations', 'experience': 'Not specified', 'skills': ['Marketing & Communications'], 'description': 'Not provided'}, 4: {'role': 'Sr. Specialist eCom Content & Promo Ops', 'experience': 'Not specified', 'skills': ['Digital'], 'description': 'Not provided'}, 5: {'role': 'Senior Manager Trade Marketing - Soccer & Running Specialty', 'experience': 'Not specified', 'skills': ['Marketing & Communications'], 'description': 'Not provided'}, 6: {'role': 'Manager eCom Content & Promo Ops', 

In [59]:
print(type(json_dict))

<class 'dict'>


In [60]:
type(json_dict)

dict

In [61]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

Techstack  \
0                React, Node.js, MongoDB   
1               Angular,.NET, SQL Server   
2      Vue.js, Ruby on Rails, PostgreSQL   
3                  Python, Django, MySQL   
4              Java, Spring Boot, Oracle   
5             Flutter, Firebase, GraphQL   
6                  WordPress, PHP, MySQL   
7                    Magento, PHP, MySQL   
8         React Native, Node.js, MongoDB   
9                  iOS, Swift, Core Data   
10       Android, Java, Room Persistence   
11             Kotlin, Android, Firebase   
12       Android TV, Kotlin, Android NDK   
13                     iOS, Swift, ARKit   
14        Cross-platform, Xamarin, Azure   
15          Backend, Kotlin, Spring Boot   
16         Frontend, TypeScript, Angular   
17    Full-stack, JavaScript, Express.js   
18  Machine Learning, Python, TensorFlow   
19               DevOps, Jenkins, Docker   

                                                Links  
0                 https://example.com/react-portfolio  
1               https://example.com/angular-portfolio  
2                   https://example.com/vue-portfolio  
3                https://example.com/python-portfolio  
4                  https://example.com/java-portfolio  
5               https://example.com/flutter-portfolio  
6             https://example.com/wordpress-portfolio  
7               https://example.com/magento-portfolio  
8          https://example.com/react-native-portfolio  
9                   https://example.com/ios-portfolio  
10              https://example.com/android-portfolio  
11       https://example.com/kotlin-android-portfolio  
12           https://example.com/android-tv-portfolio  
13               https://example.com/ios-ar-portfolio  
14              https://example.com/xamarin-portfolio  
15       https://example.com/kotlin-backend-portfolio  
16  https://example.com/typescript-frontend-portfolio  
17        https://example.com/full-stack-js-portfolio  
18            https://example.com/ml-python-portfolio  
19               https://example.com/devops-portfolio

In [62]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [63]:
links = collection.query(query_texts=["Experience in Python", "Experiance in react"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [69]:
job = json_dict

if "skills" in job:
    skills = job["skills"]
    print(skills)
else:
    print("Key 'skills' not found in job dictionary.")

Key 'skills' not found in job dictionary.


In [72]:
print(job.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])


In [73]:
for key in json_dict:
    job = json_dict[key]  
    if "skills" in job:
        print(job["skills"])
    else:
        print(f"Key 'skills' not found in job dictionary at key {key}.")

['Digital']
['Marketing & Communications']
['Marketing & Communications']
['Marketing & Communications']
['Digital']
['Marketing & Communications']
['Digital']
['Digital']
['Marketing & Communications']
['Marketing & Communications']
['Information Technology']
['Design']
['Marketing & Communications']
['Marketing & Communications']
['Marketing & Communications']
['Marketing & Communications']
['Product Development']
['Digital']
['Digital']


In [74]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Experienced E-commerce Manager Position - Leverage AtliQ's Digital Expertise for Seamless Integration

Dear Hiring Manager,

I hope this email finds you well. I am Mohan, a Business Development Executive at AtliQ, an AI & Software Consulting company specializing in facilitating the seamless integration of business processes through automated tools. We have a proven track record of empowering numerous enterprises with tailored solutions, driving scalability, process optimization, cost reduction, and enhanced overall efficiency.

Based on your recent job posting for an E-commerce Manager, I believe AtliQ's digital capabilities and expertise can significantly contribute to your team's success. Our portfolio showcases several projects highlighting our prowess in digital solutions, including:

1. Machine Learning & Python Portfolio: <https://example.com/ml-python-portfolio>
2. Python Portfolio: <https://example.com/python-portfolio>

Our experience in these areas will enable us to 